# PCA and Logistic Regression

In [ ]:
!sudo apt-get clean 

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
import findspark
findspark.init()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/My Drive/LDS9/Practice/Chapter13/'

/content/gdrive/My Drive/LDS9/Practice/Chapter13


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('PCA_sonar').getOrCreate()

In [ ]:
df = spark.read.csv('sonar_all_data.txt',inferSchema=True,header=False)

In [ ]:
df.printSchema()

root
 |-- _c0: double (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: double (nullable = true)
 |-- _c8: double (nullable = true)
 |-- _c9: double (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: double (nullable = true)
 |-- _c12: double (nullable = true)
 |-- _c13: double (nullable = true)
 |-- _c14: double (nullable = true)
 |-- _c15: double (nullable = true)
 |-- _c16: double (nullable = true)
 |-- _c17: double (nullable = true)
 |-- _c18: double (nullable = true)
 |-- _c19: double (nullable = true)
 |-- _c20: double (nullable = true)
 |-- _c21: double (nullable = true)
 |-- _c22: double (nullable = true)
 |-- _c23: double (nullable = true)
 |-- _c24: double (nullable = true)
 |-- _c25: double (nullable = true)
 |-- _c26: double (nullable = true)
 |-- _c27: double (nullable = tru

In [ ]:
df = df.withColumnRenamed("_c60","label")

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import PCA

In [ ]:
assembler = VectorAssembler(
    inputCols=['_c%d' % i for i in range(60)],
    outputCol="features")
output = assembler.transform(df)

In [ ]:
output.select("features").show(1, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                                                                                                                                                   |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Scale feature if we need

In [ ]:
standardizer = StandardScaler(withMean=True, withStd=True,
                              inputCol='features',
                              outputCol='std_features')
model = standardizer.fit(output)
output = model.transform(output)

### Label to Lable_idx

In [ ]:
indexer = StringIndexer(inputCol="label", outputCol="label_idx")
indexed = indexer.fit(output).transform(output)

In [ ]:
final_data = indexed.select(['std_features', 'label', 'label_idx'])

In [ ]:
final_data.show(3)

+--------------------+-----+---------+
|        std_features|label|label_idx|
+--------------------+-----+---------+
|[-0.3985897356694...|    R|      1.0|
|[0.70184498705605...|    R|      1.0|
|[-0.1289179854363...|    R|      1.0|
+--------------------+-----+---------+
only showing top 3 rows



### PCA

In [ ]:
# PCA

In [ ]:
# 

DenseVector([0.2035, 0.189, 0.0855, 0.0568, 0.0501, 0.0406, 0.0328, 0.0305, 0.0257, 0.0249, 0.0208, 0.019, 0.0175, 0.0154, 0.0143])

In [ ]:
#
type(percent)    

pyspark.ml.linalg.DenseVector

In [ ]:
#

0.8261807898020073

In [ ]:
#

In [ ]:
transformed.show(3)

+--------------------+-----+---------+--------------------+
|        std_features|label|label_idx|                 pca|
+--------------------+-----+---------+--------------------+
|[-0.3985897356694...|    R|      1.0|[-1.9165444107164...|
|[0.70184498705605...|    R|      1.0|[0.47896904316845...|
|[-0.1289179854363...|    R|      1.0|[-3.8499400285258...|
+--------------------+-----+---------+--------------------+
only showing top 3 rows



In [ ]:
#

### Logistic Regression

In [ ]:
train_data,test_data = final_data.randomSplit([0.8,0.2])

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
logistic = LogisticRegression(featuresCol='pca', 
                      labelCol='label_idx', 
                      predictionCol='prediction')

In [ ]:
# Fit the model to the data and call this model logisticModel
logisticModel = logistic.fit(train_data)

In [ ]:
# Create predictions for the testing data and show confusion matrix
test_model = logisticModel.transform(test_data)
test_model.groupBy('label_idx', 'prediction').count().show()

+---------+----------+-----+
|label_idx|prediction|count|
+---------+----------+-----+
|      1.0|       1.0|   14|
|      0.0|       1.0|    6|
|      1.0|       0.0|    8|
|      0.0|       0.0|   13|
+---------+----------+-----+



In [ ]:
# Calculate the elements of the confusion matrix
TN = test_model.filter('prediction = 0 AND label_idx = prediction').count()
TP = test_model.filter('prediction = 1 AND label_idx = prediction').count()
FN = test_model.filter('prediction = 0 AND label_idx != prediction').count()
FP = test_model.filter('prediction = 1 AND label_idx != prediction').count()

In [ ]:
# Calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)
print('precision = {:.2f}\nrecall    = {:.2f}'.format(precision, recall))

precision = 0.70
recall    = 0.64


In [ ]:
acc =(TP+TN)/test_model.count()
acc

0.6585365853658537